In [1]:
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from src.DataLoader import Room,Receiver

INFO - Logger configured


In [2]:
#Map Data
MapDataPath="data/cartePalaiseau.csv"
map_gdf=gdfLoader(MapDataPath)
#Receiver Data
ReceiverDataPath="data/carteModulePalaiseau.csv"
receiver_gdf=ReceiverLoader(ReceiverDataPath)

In [3]:
receiver_gdf

,macModule,geometry,coordinatesEPSG3857
0,A8032A31204E,POINT (2.20079 48.71316),POINT (244990.444 6226329.680)
1,C45BBE39F42A,POINT (2.20110 48.71315),POINT (245025.333 6226329.187)
2,C45BBE39F9AE,POINT (2.20120 48.71314),POINT (245035.947 6226326.828)
3,A8032A311FAA,POINT (2.20072 48.71317),POINT (244983.036 6226331.224)
4,C45BBE39F56A,POINT (2.20117 48.71318),POINT (245033.645 6226333.296)
5,C45BBE37B346,POINT (2.20087 48.71315),POINT (244999.412 6226328.251)
6,C45BBE386952,POINT (2.20098 48.71316),POINT (245011.849 6226330.207)
7,C45BBE386362,POINT (2.20094 48.71314),POINT (245007.682 6226326.760)
8,C45BBE384CD2,POINT (2.20093 48.71317),POINT (245006.234 6226331.144)
9,C45BBE39870E,POINT (2.20106 48.71315),POINT (245020.561 6226328.967)


In [4]:
map_gdf

,ID_element,nom,geometry,coordinatesEPSG3857
0,21205,4A101,"POLYGON ((2.19977 48.71281, 2.19979 48.71287, ...","POLYGON ((244877.665 6226270.774, 244879.456 6..."
1,21264,4A413,"POLYGON ((2.20045 48.71307, 2.20046 48.71311, ...","POLYGON ((244952.833 6226315.319, 244953.915 6..."
2,21265,4A409,"POLYGON ((2.20043 48.71301, 2.20044 48.71304, ...","POLYGON ((244951.228 6226305.055, 244951.825 6..."
3,21266,,"POLYGON ((2.20050 48.71315, 2.20055 48.71315, ...","POLYGON ((244958.446 6226328.379, 244963.992 6..."
4,21267,,"POLYGON ((2.20055 48.71317, 2.20059 48.71316, ...","POLYGON ((244964.552 6226330.920, 244968.583 6..."
...,...,...,...,...
196,21994,4A249,"POLYGON ((2.20069 48.71252, 2.20073 48.71251, ...","POLYGON ((244979.415 6226221.425, 244984.035 6..."
197,21995,,"POLYGON ((2.20140 48.71297, 2.20138 48.71298, ...","POLYGON ((245058.541 6226298.509, 245056.217 6..."
198,21996,,"POLYGON ((2.20109 48.71319, 2.20110 48.71324, ...","POLYGON ((245024.347 6226335.828, 245025.612 6..."
199,21997,,"POLYGON ((2.20116 48.71308, 2.20129 48.71307, ...","POLYGON ((245031.845 6226316.774, 245046.289 6..."


In [5]:
from src.DataLoader import room_list_generator,room_r_tree_generator
room_list=room_list_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_list)


In [6]:
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(receiver_gdf,room_list,room_r_tree)


In [7]:
Threshold1=-45
Threshold2=-70

In [8]:
from src.DataLoader import Receiver
from shapely.geometry import Point,Polygon

#need in the class: receiver_dict
def zone_selection(receiver:Receiver,rssi:int)->Polygon:
    """
    Return the predicted zone for the selected receiver receiving the rssi 
    
    Parameters
    ---
    Input:
    receiver: Receiver
    rssi: int
    
    Output:
    Polygon
    """
    if rssi>Threshold1:
        return receiver.room.cartesian_polygon
    if Threshold1< rssi < Threshold2:
        pass
        
        #intersect with all room within a certain radius        
    

In [9]:
receiver_dict['A8032A31204E']


Receiver(room=Room(uid=21884, name='4A443', spherical_polygon=<POLYGON ((2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48....>, cartesian_polygon=<POLYGON ((244983.945 6226325.214, 244984.89 6226331.352, 244993.154 6226329...>), spherical_point=<POINT (2.201 48.713)>, cartesian_point=<POINT (244990.444 6226329.68)>)

In [10]:
def test_case1(receiver_dict):
    receiver=receiver_dict['A8032A31204E']
    rssi=-40
    zone=zone_selection(receiver,rssi)
    assert zone==receiver.room.cartesian_polygon,f'Error: {zone_selection(receiver,rssi)} should be {receiver.room.cartesian_polygon}'
test_case1(receiver_dict)
def test_full_treshold(receiver_dict):
    for receiver in receiver_dict.values():
        rssi=-40
        zone=zone_selection(receiver,rssi)
        assert zone==receiver.room.cartesian_polygon,f'Error: {zone_selection(receiver,rssi)} should be {receiver.room.cartesian_polygon}'
test_full_treshold(receiver_dict)

In [11]:
from src.visualization import MapCreation

map_object=MapCreation(receiver_dict,room_list)


[48.713157751163116, 2.2007866061683496]
[48.71315483004127, 2.2011000149556117]
[48.71314084307316, 2.201195363079581]
[48.713166904690254, 2.2007200581389768]
[48.71317918188483, 2.2011746828046217]
[48.71314928147962, 2.200867161154747]
[48.71316087476933, 2.200978883869687]
[48.713140444154725, 2.2009414505789455]
[48.71316642923294, 2.2009284504732873]
[48.71315352665187, 2.2010571514186674]
[48.71317804320248, 2.2008387406309424]
[48.71318498422266, 2.2007622901244153]
[48.71317445007082, 2.200638986440939]
[48.713203074448835, 2.200612881651723]
[48.7131327761428, 2.201013030945527]
[48.71317608031125, 2.200493985751738]
[48.713174265469796, 2.200573795873236]
[48.71318388680103, 2.2004270113114988]
[48.71311931767969, 2.2011441558369036]
[48.71319455060883, 2.2006869316821076]
[48.71313855206642, 2.2015450224280353]
[48.7131157216373, 2.2015825062990193]
[48.7131250573373, 2.2014675736427303]
[48.71313235776498, 2.201395153999328]
[48.7131642141643, 2.2013659849762917]
[48.7131

In [12]:
map_object